In [1]:
import numpy as np
import seaborn as sns

In [66]:
from scipy import stats

In [124]:
# Number of clusters in each cohorts
N = 5000

# The average number of samples in each cluster follows a poisson distribution with mean lambda
mean_samples = 5

# Number of simulations
n_iterations = 10000

# Conversion Rate - Follow a Beta Distribution
alpha = 1
beta = 3

In [125]:
np.random.seed(888)

### Type One Error  

#### Simulation under cluster level, randomization unit is same as the analysis unit  (Correct way !)

In [126]:
# Calculate the metric in cluster level
# 5% significance level
false_positive = 0
for i in range(n_iterations):
    # conversion rate p in each cluster are from the same beta distribution in control and test group
    control_cluster_p = np.random.beta(alpha, beta, N)
    treatment_cluster_p = np.random.beta(alpha, beta, N)

    # number of samples in each cluster, plus one to make sure the samples are greater than 1
    control_samples_per_cluster = np.random.poisson(mean_samples, N) + 1
    treatment_samples_per_cluster = np.random.poisson(mean_samples, N) + 1

    # number of converted samples in each cluster
    control_converted = np.random.binomial(
        n=control_samples_per_cluster, p=control_cluster_p)

    treatment_converted = np.random.binomial(
        n=treatment_samples_per_cluster, p=treatment_cluster_p)

    # conversion in each cluster
    control_cluster_conversion = control_converted / control_samples_per_cluster
    treatment_cluster_conversion = treatment_converted / treatment_samples_per_cluster

    # perform t-test
    control_mean, var_c = np.mean(control_cluster_conversion),  np.var(
        control_cluster_conversion)
    treatment_mean, var_t = np.mean(treatment_cluster_conversion), np.var(
        treatment_cluster_conversion)
    t_score = (treatment_mean - control_mean) / np.sqrt(var_c / N + var_t / N)

    if (t_score >= 1.96) or (t_score <= -1.96):
        false_positive += 1

print("Type One Error : {}".format(false_positive / n_iterations))

Type One Error : 0.051


#### Simulation under samples level, aalysis unit is more granular than the randomization unit (Wrong way !)

In [127]:
# Calculate the metric in sample level
# 5% significance level
false_positive = 0
for i in range(n_iterations):
    # conversion rate p in each cluster are from the same beta distribution in control and test group
    control_cluster_p = np.random.beta(alpha, beta, N)
    treatment_cluster_p = np.random.beta(alpha, beta, N)

    # number of samples in each cluster, plus one to make sure the samples are greater than 1
    control_samples_per_cluster = np.random.poisson(mean_samples, N) + 1
    treatment_samples_per_cluster = np.random.poisson(mean_samples, N) + 1

    # number of converted samples in each cluster
    control_converted = np.random.binomial(
        n=control_samples_per_cluster, p=control_cluster_p)

    treatment_converted = np.random.binomial(
        n=treatment_samples_per_cluster, p=treatment_cluster_p)

    # conversion in each cohort
    control_cluster_conversion = sum(control_converted) / sum(control_samples_per_cluster)
    treatment_cluster_conversion = sum(treatment_converted) / sum(treatment_samples_per_cluster)

    # perform two proportion z-test
    control_mean, var_c = control_cluster_conversion,  control_cluster_conversion * (1 - control_cluster_conversion)
    treatment_mean, var_t = treatment_cluster_conversion, treatment_cluster_conversion * (1 - treatment_cluster_conversion)
    z_score = (treatment_mean - control_mean) / np.sqrt(var_c / sum(control_samples_per_cluster) + var_t / sum(treatment_samples_per_cluster))

    if (z_score >= 1.96) or (z_score <= -1.96):
        false_positive += 1

print("Type One Error : {}".format(false_positive / n_iterations))

Type One Error : 0.1852


In [128]:
### Woooo !  False positive is way higher than what we expected ! The reason is 
### In the above simulation, We draw samples from different clusters 
### If two samples are coming from the same clusters, they are no longer independent to each other !!!
### when x, y are from the same cluster, P(x|y) != p(x)

#### Use delta method to correct for the variance estimation (??)